Generates a single JSON file from a GitHub repo so that it can be used as a custom GPT resource.

In [1]:
import { read } from "https://deno.land/x/streaming_zip/read.ts";
import { Buffer } from "jsr:@std/streams"

async function downloadRepoAsZip(owner: string, repo: string, branch: string = 'main', token?: string): Promise<Response> {
  const url = `https://api.github.com/repos/${owner}/${repo}/zipball/${branch}`;
  const headers: HeadersInit = {};

  if (token) {
    headers['Authorization'] = `token ${token}`;
  }

  const response = await fetch(url, { headers });
  if (response.status === 200) {
    return response;
  } else {
    throw new Error(`Failed to download repository: ${response.status}`);
  }
}

async function processZipStream(zipStream: ReadableStream<Uint8Array>): Promise<unknown[]> {
  const files: unknown[] = [];

  for await (const entry of read(zipStream)) {
    if (entry.type === "file") {
      const buffer = new Buffer();
      await entry.body.stream().pipeTo(buffer.writable);
      const data = new TextDecoder().decode(buffer.bytes({ copy: false }));
      const path = entry.name.split('/').slice(1).join('/');
      files.push({ path, data });
    }
  }

  return files;
}

async function generateGptRepo(owner: string, repoName: string, branch: string, token?: string) {
  try {
    // Download the repository as a zip file
    const response = await downloadRepoAsZip(owner, repoName, branch, token);

    // Process files from zip stream
    const files = await processZipStream(response.body!);

    // Write output to a file
    await Deno.writeTextFile(`${repoName}-repo.json`, JSON.stringify(files, null, 2));
  } catch (error) {
    console.error(`Error: ${error.message}`);
  }
}

await generateGptRepo("ggpwnkthx", "deno-jupyter", "key-value");